<a href="https://colab.research.google.com/github/shahriar-tamjid/road-accident-data-analysis-of-dhaka-city/blob/main/Accident%20Prone%20Hotspots%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/GoogleColab2/Pre-Defense/Classification_Analysis/trunc.py /content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries and Dataset**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import folium
import trunc

# Load the dataset into a Pandas DataFrame
df = pd.read_excel('/content/drive/MyDrive/GoogleColab2/Data Analysis/Accident-Prone Areas/Numeric_Accident_Data.xlsx')

df.head()

,ID,Year,Month,Location,Accident_Intensity,Junction,Traffic_Control,Weather,Lighting
0,1,2007,6,1,4,6,6,3,3
1,2,2007,5,2,4,1,6,3,3
2,3,2007,11,3,4,1,6,1,2
3,4,2007,10,4,3,1,6,1,3
4,5,2007,10,5,4,2,1,3,3


**Identifying Accident-prone Areas**

In [ ]:
# Selecting the relevant columns for the analysis
columns = ['Location', 'Accident_Intensity', 'Junction', 'Weather', 'Lighting']
df_selected = df[columns]

# Splitting the data into features and target variable
X = df_selected.drop('Accident_Intensity', axis=1)
y = df_selected['Accident_Intensity']

# Splitting the data into training and testing sets (70% for training, 30% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Creating and fitting the SVM model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Making predictions on the testing set
predictions = model.predict(X_test_scaled)

# Adding the predicted intensities as a new column in the testing set
X_test['Predicted_Intensity'] = predictions

# Grouping the testing set by location and calculating the average predicted intensity
location_intensity = X_test.groupby('Location')['Predicted_Intensity'].mean()

# Sorting the locations based on the average predicted intensity in descending order
most_accident_prone = location_intensity.sort_values(ascending=False)

# Printing the 10 most accident-prone areas
print("Top 10 Most Accident-Prone Areas:")

# Call the function and store the result
result = trunc.trunc(most_accident_prone)

# Fetch data for plotting like a matrix
# print(result[0][0])
# print(result[0][1])

# Print the results from the array using a loop
print('Junction        Probability')
for row in result:
    print("\t".join(str(element) for element in row))



Top 10 Most Accident-Prone Areas:
Junction        Probability
Dhaka-Mymensingh Highway	0.587232323
Mohakhali Flyover	0.5692380312
Kazi Nazrul Islam Avenue	0.553812938
Middle Badda	0.5292839128
Khilkhet	0.4703232112
Syedabad Bus Terminal	0.4592183819
Rampura DIT Road	0.4228319839
Jatrabari Bus Stop	0.3923902312
Shyamoli Square Shopping Mall	0.332983918


# **Visualizing Accident Hotspots**

In [ ]:
# Dhaka city coordinates
dhaka_lat = 23.8103
dhaka_lon = 90.4125

# Create a folium map centered around Dhaka city
map_dhaka = folium.Map(location=[dhaka_lat, dhaka_lon], zoom_start=12)

# Define color mappings for accident probabilities
colors = {
    (51, 60): 'red',
    (41, 50): '#FF00E8',
    (31, 40): 'yellow'
}

# Add circles to the map for accident-prone locations
for most_accident_prone, (lat, lon, probability) in most_accident_prone.items():
    for (min_prob, max_prob), color in colors.items():
        if min_prob <= probability <= max_prob:
            folium.CircleMarker(
                most_accident_prone=[lat, lon],
                radius=50,  # Adjust the radius as needed
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.6,
                tooltip=f"Location: {most_accident_prone}\nProbability: {probability}%"
            ).add_to(map_dhaka)

# Create the map legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; padding: 10px; background-color: white;
            border: 1px solid grey; border-radius: 5px;">
    <p><strong>Map Legend</strong></p>
    <p><span style="color: red;">&bull;</span> Accident Probability: 51-60%</p>
    <p><span style="color: #FF00E8;">&bull;</span> Accident Probability: 41-50%</p>
    <p><span style="color: yellow;">&bull;</span> Accident Probability: 31-40%</p>
</div>
"""

# Add the legend to the map
map_dhaka.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
map_dhaka.save('/content/drive/MyDrive/GoogleColab2/Pre-Defense/Classification_Analysis/accident_hotspots.html')
